### Critical note:
Topic modelling was optimised on the NON-DEDUPLICATED corpus; however, this resulted in more "reasonable" topics than implemented on the deduplicated one, so the model used below comes from the full cleaned but not deduplicated corpus. It is being APPLIED in this document to the deduplicated corpus.

The optimal model on the deduplicated corpus had 18 topics and is also provided.

In [1]:
import pathlib
from utils import get_projectpaths
(projectroot, rawdatapath, cleandatapath, processeddatapath) = get_projectpaths()
import re
import numpy as np
import pandas as pd
# silence annoying warning
pd.options.mode.chained_assignment = None  # default='warn'
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
# %matplotlib inline
# Plotting tools
import pyLDAvis
# may need more from here https://stackoverflow.com/questions/66759852/no-module-named-pyldavis
import pyLDAvis.gensim_models as gensimvis
# Enable logging for gensim - optional
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import pickle
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# spacy for lemmatization
import spacy
from pprint import pprint

In [2]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatize(texts, allowed_postags=['PROPN','NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations



# WORK BLOCK ------------------------------------------------
# %% load data and obesity names
corpusdf = pd.read_pickle(processeddatapath/'corpusdf_deduped_by_source.pickle')



# %% 
# Convert body to list
bodies = corpusdf.body.values.tolist()
# Remove new lines
bodies = [re.sub('\s+', ' ', sent) for sent in bodies]
# Remove single quotes
bodies = [re.sub("\'", "", sent) for sent in bodies]
# Remove double quotes
bodies = [re.sub('"', "", sent) for sent in bodies]
bodies_words = list(sent_to_words(bodies))
# Remove Stop Words
bodies_words_nostops = remove_stopwords(bodies_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun/propn, adj, vb, adv
bodies_lemmatized = lemmatize(bodies_words_nostops, allowed_postags=['PROPN','NOUN', 'ADJ', 'VERB', 'ADV'])



# %% Create Dictionary
corpusdict = corpora.Dictionary(bodies_lemmatized)
# print how many words are in the dictionary
# gensim mem usage will be 24 * num_topics * this
print(corpusdict)


# Term Document Frequency
corpus = [corpusdict.doc2bow(text) for text in bodies_lemmatized]


# %%
lda_model =  gensim.models.LdaModel.load(str(processeddatapath/'topicmodels/full_m_17_model'))


# %% Print the Keyword in the topics
pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]


# %% Compute Perplexity
# Perplexity:  -8.685147984244903
# print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.




Dictionary(95737 unique tokens: ['accord', 'adult', 'ask', 'authoritie', 'balance']...)
[(0,
  '0.010*"people" + 0.007*"year" + 0.006*"time" + 0.006*"work" + 0.005*"say" + '
  '0.005*"need" + 0.005*"many" + 0.005*"well" + 0.005*"do" + 0.005*"life"'),
 (1,
  '0.022*"study" + 0.016*"say" + 0.013*"people" + 0.011*"find" + '
  '0.011*"researcher" + 0.010*"research" + 0.009*"university" + 0.008*"brain" '
  '+ 0.008*"exercise" + 0.008*"eat"'),
 (2,
  '0.023*"say" + 0.021*"patient" + 0.011*"pain" + 0.010*"treatment" + '
  '0.009*"liver" + 0.009*"condition" + 0.008*"surgery" + 0.008*"year" + '
  '0.008*"dr" + 0.008*"replacement"'),
 (3,
  '0.025*"food" + 0.018*"diet" + 0.016*"eat" + 0.016*"fat" + 0.013*"sugar" + '
  '0.011*"vitamin" + 0.011*"fruit" + 0.011*"say" + 0.009*"low" + '
  '0.008*"cancer"'),
 (4,
  '0.015*"get" + 0.014*"say" + 0.013*"dog" + 0.011*"do" + 0.009*"go" + '
  '0.008*"exercise" + 0.008*"people" + 0.008*"time" + 0.008*"day" + '
  '0.007*"run"'),
 (5,
  '0.008*"get" + 0.007*"s

In [3]:
# %%
import pyLDAvis.gensim_models as gensim_models
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensim_models.prepare(lda_model, corpus, corpusdict)
vis

/Users/darya/anaconda3/envs/obesity/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/darya/anaconda3/envs/obesity/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/darya/anaconda3/envs/obesity/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/darya/anaconda3/envs/obesity/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.079048 -0.164893       1        1  13.786250
11    -0.123616 -0.012871       2        1  10.233227
0     -0.052773 -0.073155       3        1   9.724305
17     0.039484  0.132805       4        1   7.639146
16     0.054568 -0.023763       5        1   7.065412
14     0.002630  0.041089       6        1   6.869694
10     0.005648 -0.153577       7        1   6.117060
3      0.140225  0.084111       8        1   5.433911
9      0.080592  0.075646       9        1   5.247294
4      0.079241 -0.111880      10        1   5.119314
1      0.113104  0.067810      11        1   5.043833
15    -0.038541 -0.111489      12        1   4.337839
8     -0.123440 -0.089597      13        1   3.226865
12    -0.363184  0.163437      14        1   3.151463
13     0.060277  0.031552      15        1   2.047433
7      0.120425 -0.030883      16        1   2.013468
2      0.041235  0.069561      17        1   1.622101
6      0.043175  0.106098      18        1   1.321385, topic_info=         Term           Freq          Total Category  logprob  loglift
715   service   22913.000000   22913.000000  Default  30.0000  30.0000
377     child   36463.000000   36463.000000  Default  29.0000  29.0000
43        say  126026.000000  126026.000000  Default  28.0000  28.0000
1843       mr   17491.000000   17491.000000  Default  27.0000  27.0000
465    weight   25866.000000   25866.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
343       use     563.582582   24119.459837  Topic18  -5.3362   0.5700
166       get     578.171333   41144.474616  Topic18  -5.3106   0.0615
393   develop     481.240330    7340.610986  Topic18  -5.4941   1.6017
23    medical     469.230079   10582.749389  Topic18  -5.5194   1.2106
249    people     473.335837   56052.674370  Topic18  -5.5107  -0.4478

[1609 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
16390      1  0.123506  abbott
16390      2  0.698784  abbott
16390      3  0.144957  abbott
16390      4  0.001300  abbott
16390      6  0.002600  abbott
...      ...       ...     ...
1486       2  0.005516    zinc
1486       3  0.005516    zinc
1486       8  0.976283    zinc
1486      10  0.005516    zinc
14258      2  0.975217  zoning

[10558 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 12, 1, 18, 17, 15, 11, 4, 10, 5, 2, 16, 9, 13, 14, 8, 3, 7])